rightmove web scrapper


In [6]:
# import all neccessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [7]:
rm_salesurl = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index={index}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
rm_renturl = "https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E87490&index={index}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [10]:

def connect_page(url, start, stop, increment):
    try:
        pages = []  # List to store all the pages

        for i in range(start, stop, increment):
            # Format the URL with the updated index value
            page_html = url.format(index=i)

            response = requests.get(page_html)
            if response.status_code == 200:
                content = BeautifulSoup(response.content, 'html.parser')
                page = BeautifulSoup(content.prettify(), 'html.parser')
                pages.append(page)  # Add the page to the list
                

            else:
                print("Error:", response.status_code)
                # Handle the error condition if needed

        return pages  # Return the list of pages

    except requests.ConnectionError as e:
        print("Connection Error:", str(e))

        

def get_data(pages:'html page', transaction_type:str, source:str):

    addresses = []
    bedrooms = []
    bathrooms = []
    descriptions = []
    property_types = []
    prices = []
    locations = []
    agents = []
    listing_urls = []
    date_type = []
    dates = []

    for page in pages:
        
        address_elements = page.find_all('address', class_='propertyCard-address')
        for item in address_elements:
            address = item.get_text(strip=True) if item else ""
            addresses.append(address)
        
        bedroom_elements = page.find_all('span', {'class': 'no-svg-bed-icon bed-icon seperator'})
        bedroom = " "
        for item in bedroom_elements:
            bedroom = item['title'] if item else ""
            bedrooms.append(bedroom)

        bathroom_elements = page.find_all('span', class_='no-svg-bathroom-icon bathroom-icon seperator')
        bathroom=" "
        for item in bathroom_elements:
            bathroom = item.get_text(strip=True) if item else ""
            bathrooms.append(bathroom)


        description_elements = page.find_all('span', itemprop="description")
        for item in description_elements:
            description = item.get_text(strip=True) if item else ""
            descriptions.append(description)

        property_type_elements = page.find_all(class_="propertyCard-title")
        for item in property_type_elements:
            property_type_text = item.get_text(strip=True) if item else ""
            property_type = property_type_text.split(",")[0].strip() if property_type_text else ""
            property_types.append( property_type)

        price_elements = page.find_all('div', class_="propertyCard-priceValue")
        for item in price_elements:
            price = item.get_text(strip=True) if item else ""
            prices.append(price)

        location_elements = page.find_all('address', class_="propertyCard-address")
        for item in location_elements:
            address = item.get_text(strip=True) if item else ""
            location = address.split(",")[-1].strip() if address else ""
            locations.append(location)

        
        agent_elements = page.find_all('a', {'class': 'propertyCard-branchLogo-link'})
        for item in agent_elements:
            agent = item['title'] if item else ""
            agents.append(agent)

        listing_url_elements = page.find_all('a', {'class': 'propertyCard-branchLogo-link'})
        for item in listing_url_elements:
            listing_url = 'https://www.rightmove.co.uk' + item['href'] if item else ""
            listing_urls.append(listing_url)

        date_added_elements = page.find_all('span', class_="propertyCard-contactsAddedOrReduced")
        for item in date_added_elements:
            datetype = item.get_text(strip=True).split(" on ")[0].strip() if item else ""
            date_added = item.get_text(strip=True).split(" on ")[-1].strip() if item else ""
            dates.append(date_added)
            date_type.append(datetype)

        listing_source = source
        transaction = transaction_type
  
        data = pd.DataFrame()
        data['address']= addresses
        data['address']= addresses 
        # data['bedroom']=bedrooms 
        # data['bathroom']=bathrooms 
        data['description']=descriptions 
        data['property_type']=property_types 
        data['price']=prices 
        data['location']=locations 
        # data['agent']=agents
        data['transaction_type'] = transaction
        data['listing_source'] = listing_source
        # data['listing_url']=listing_urls 
        data['date_type']=date_type 
        data['date']=dates 


    return data


In [11]:
# Set the range of indexes you want to loop through
start_index = 0
stop_index = 24
increment = 24

# Call the function with the URL and index range
page = connect_page(rm_salesurl, start_index, stop_index, increment)
df = get_data(page,'sales', 'rightmove') 
len(df)

25

In [12]:
df

,address,description,property_type,price,location,transaction_type,listing_source,date_type,date
0,"The Driftway, Bow","Price Guide £425,000 - £450,000 Positioned wit...",1 bedroom apartment for sale,"£425,000",Bow,sales,rightmove,Reduced,12/05/2023
1,"One Hyde Park, Knightsbridge",An exceptional exclusive five bedroom apartmen...,5 bedroom apartment for sale,"£60,000,000",Knightsbridge,sales,rightmove,Added,22/12/2022
2,"Lygon Place, Belgravia, SW1W",Ref. LOB0798 - Set behind a gated Belgravia dr...,7 bedroom house for sale,"£45,000,000",SW1W,sales,rightmove,Added,21/11/2022
3,"Mayfair Freehold House, Park Lane Area, W1K",This stunning 12 bedroom (including 3 staff be...,12 bedroom house for sale,"£45,000,000",W1K,sales,rightmove,Added,31/12/2022
4,"Pitt Street, London, W8","A rare opportunity to own this unique, contemp...",7 bedroom detached house for sale,"£44,000,000",W8,sales,rightmove,Added,20/02/2023
5,"Pitt Street, Kensington, W8",Ref. LOB0625 - This striking and unique house ...,6 bedroom house for sale,"£44,000,000",W8,sales,rightmove,Added,21/02/2023
6,"Belgravia Gate, Grosvenor Crescent, Belgravia,...",Belgravia Gate benefits from a truly world-cla...,6 bedroom property for sale,"£43,700,000",SW1X,sales,rightmove,Added,02/05/2023
7,"Brick Street, Mayfair W1J",A truly exeptional low- built 7 bedroom mansio...,7 bedroom house for sale,"£35,000,000",Mayfair W1J,sales,rightmove,Added,22/02/2023
8,"South Street, Mayfair W1K",Considered to be the archetypical London mansi...,6 bedroom house for sale,"£35,000,000",Mayfair W1K,sales,rightmove,Added,09/03/2023
9,"Royal Westminster, London",A very rare find this luxurious circa 11000 SQ...,7 bedroom detached house for sale,"£35,000,000",London,sales,rightmove,Added,04/05/2023


In [ ]:
def get_data(page, transaction_type, source):

    addresses = []
    bedrooms = []
    bathrooms = []
    descriptions = []
    property_types = []
    prices = []
    locations = []
    agents = []
    listing_urls = []
    dates = []


    address_elements = page.find_all('address', class_='propertyCard-address')
    for item in address_elements:
        address = item.get_text(strip=True) if item else ""
        addresses.append(address)

    bedroom_elements = page.find_all('span', {'class': 'no-svg-bed-icon bed-icon seperator'})
    bedroom = " "
    for item in bedroom_elements:
        bedroom = item['title'] if item else ""
        bedrooms.append(bedroom)

    bathroom_elements = page.find_all('span', class_='no-svg-bathroom-icon bathroom-icon seperator')
    bathroom=" "
    for item in bathroom_elements:
        bathroom = item.get_text(strip=True) if item else ""
        bathrooms.append(bathroom)

    description_elements = page.find_all('span', itemprop="description")
    for item in description_elements:
        description = item.get_text(strip=True) if item else ""
        descriptions.append(description)

    property_type_elements = page.find_all(class_="propertyCard-title")
    for item in property_type_elements:
        property_type_text = item.get_text(strip=True) if item else ""
        property_type = property_type_text.split(",")[0].strip() if property_type_text else ""
        property_types.append( property_type)

    price_elements = page.find_all('div', class_="propertyCard-priceValue")
    for item in price_elements:
        price = item.get_text(strip=True) if item else ""
        prices.append(price)

    location_elements = page.find_all('address', class_="propertyCard-address")
    for item in location_elements:
        address = item.get_text(strip=True) if item else ""
        location = address.split(",")[-1].strip() if address else ""
        locations.append(location)

    agent_elements = page.find_all('a', {'class': 'propertyCard-branchLogo-link'})
    for item in agent_elements:
        agent = item['title'] if item else ""
        agents.append(agent)

    # listing_source = str(source)
    # transaction = transactions.append(str(transaction_type))

    listing_url_elements = page.find_all('a', {'class': 'propertyCard-branchLogo-link'})
    for item in listing_url_elements:
        listing_url = 'https://www.rightmove.co.uk' + item['href'] if item else ""
        listing_urls.append(listing_url)

    date_added_elements = page.find_all('span', class_="propertyCard-contactsAddedOrReduced")
    for item in date_added_elements:
        date_added = item.get_text(strip=True) if item else ""
        dates.append(date_added)


    data = pd.DataFrame()
    # data['transaction'] = 
    data['address']= addresses
    data['address']=addresses 
    # data['bedroom']=bedrooms 
    # data['bathroom']=bathrooms 
    data['description']=descriptions 
    data['property_type']=property_types 
    data['price']=prices 
    data['location']=locations 
    # data['agent']=agents 
    # data['listing_url']=listing_urls 
    data['date_added']=dates 


    return data
